# PostgreSQL installation & data loading: guidelines

## Installation of PostgreSQL

### Linux

* Install PostgreSQL:
https://www.digitalocean.com/community/tutorials/how-to-install-and-use-postgresql-on-ubuntu-16-04

        sudo apt-get update
        sudo apt-get install postgresql postgresql-contrib


* Install DBeaver (if a user interface is wanted): 
    * https://dbeaver.io/
    * https://computingforgeeks.com/install-dbeaver-on-ubuntu-18-04-ubuntu-16-04-debian-9/

            wget -O - https://dbeaver.io/debs/dbeaver.gpg.key | sudo apt-key add -
            echo "deb https://dbeaver.io/debs/dbeaver-ce /" | sudo tee /etc/apt/sources.list.d/dbeaver.list

            sudo apt update
            sudo apt -y  install dbeaver-ce

* Execute PostgreSQL and create a user (password on demand):
     
        sudo -i -u postgres
        createuser -P -s -e <user_name>

* The connection will be with:
    * host: _localhost_
    * user: _<user_name>_
    * pwd: _<pwd_generated>_


* Create test database:

        sudo -u postgres createdb <dbname>

### Mac

* https://www.postgresql.org/download/macosx/
* https://www.robinwieruch.de/postgres-sql-macos-setup/

### Windows

https://www.postgresql.org/download/windows/


## Data loading

Once the DB and a user are created, you should change the **credentials** and the **data_path** below and it should be working!

In [ ]:
# For installing packages from Jupyter notebook
# import sys
# !{sys.executable} -m pip install psycopg2

# To install requirements.txt
# !{sys.executable} -m pip install -r requirements.txt

In [ ]:
import glob
import os
import pandas as pd
from pathlib import Path
import psycopg2

In [ ]:
# Credentials to connect to DB 
# (this should not be placed inside the code!)
host = 'localhost'
user = ...
pwd = ...
db = ...

In [ ]:
# Path where the data to load into the DB is found
data_path = ...

In [ ]:
# Connection to DB
conn = psycopg2.connect(dbname=db, user=user, password=pwd, host=host)

In [ ]:
def create_tables(host, dbname, user, pwd):
    """ Create tables in the PostgreSQL database"""
    
    commands = (
        """
        CREATE TABLE IF NOT EXISTS people_jobs (
            person_id SMALLINT NOT NULL,
            job_id SMALLINT NOT NULL,
            CONSTRAINT people_jobs_pkey PRIMARY KEY (person_id, job_id)
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS jobs (
            job_id SMALLINT PRIMARY KEY,
            job_name TEXT NOT NULL
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS housing (
            house_id SMALLINT PRIMARY KEY,
            neighborhood TEXT NOT NULL,
            locality TEXT NOT NULL
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS people (
                person_id SMALLINT PRIMARY KEY,
                name VARCHAR(50) NOT NULL,
                age SMALLINT,
                gender VARCHAR(1),
                house_id SMALLINT,
                fav_music_gndr TEXT,
                debt_balance REAL
        )
        """)
    conn = None
    try:
        # Connect to the PostgreSQL server
        conn = psycopg2.connect(dbname=db, user=user, password=pwd, host=host)
        cur = conn.cursor()
        
        # Create table one by one
        for command in commands:
            cur.execute(command)
        
        # Close communication with the PostgreSQL database server
        cur.close()
        
        # Commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
# Create tables
create_tables(host=host, dbname=db, user=user, pwd=pwd)

In [ ]:
# Transform data_path into Path, if it is a string
if type(data_path) == str:
    data_path = Path(data_path)

# Load data into brand new DB
cur = conn.cursor()

csv_files = [f for f in data_path.glob('*.csv')]

for file in csv_files:
    
    df = pd.read_csv(file, sep=";")
    tab_name = file.stem.replace('tab_', '')
    
    print("Loading table", tab_name)
    
    try:
        with open(file, 'r') as f:
            next(f)  # Skip the header row.
            cur.copy_from(f, tab_name, sep=';', null='None')
            
    except Exception as e:
        print(e)
        conn.commit()
        cur.close()
        cur = conn.cursor()
        continue
    
conn.commit()
cur.close()

## DB checks

In [ ]:
delete_tabs = True

In [ ]:
if delete_tabs:
    cur = conn.cursor()

    for tab in ['jobs', 'people_jobs', 'housing', 'people']:
        query = f"""
        DROP TABLE {tab} 
        """

        cur.execute(query)

    # Commit the changes
    conn.commit()

In [ ]:
query = """
SELECT * FROM people
"""

In [ ]:
people = pd.read_sql_query(query, conn)
people.head()